In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import config 
import sql_con
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import csv

In [2]:
# get module variables
ROOT_DIR = config.ROOT_DIR
select_records = sql_con.select_records
insert_records = sql_con.insert_records
update_records = sql_con.update_records
conn_odbc = sql_con.conn_odbc
read_contents = sql_con.read_contents

In [3]:
# make request to cryptocompare api to get historical data for bitcoin quote prices in USD
url = "https://min-api.cryptocompare.com/data/histoday"
parameters = {
  "fsym":"BTC",
  "tsym":"USD",
  "allData":"true"
}
headers = {
  "authorization": f"Apikey {config.API_KEY}",
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  res_json = json.loads(response.text)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

# res1 = requests.get(url, headers=headers)

In [4]:
# get data from json response and normalize it into a pandas dataframe
data = res_json["Data"]
data = pd.json_normalize(data)

In [5]:
data.head()

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,1279324800,0.04951,0.04951,0.04951,20.00,0.9902,0.04951,direct,
1,1279411200,0.08585,0.05941,0.04951,75.01,5.0920,0.08584,direct,
2,1279497600,0.09307,0.07723,0.08584,574.00,49.6600,0.08080,direct,
3,1279584000,0.08181,0.07426,0.08080,262.00,20.5900,0.07474,direct,
4,1279670400,0.07921,0.06634,0.07474,575.00,42.2600,0.07921,direct,


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4637 entries, 0 to 4636
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   time              4637 non-null   int64  
 1   high              4637 non-null   float64
 2   low               4637 non-null   float64
 3   open              4637 non-null   float64
 4   volumefrom        4637 non-null   float64
 5   volumeto          4637 non-null   float64
 6   close             4637 non-null   float64
 7   conversionType    4637 non-null   object 
 8   conversionSymbol  4637 non-null   object 
dtypes: float64(6), int64(1), object(2)
memory usage: 326.2+ KB


In [7]:
# ingest data into excel file
data.to_excel(rf"{ROOT_DIR}/data/btc_price.xlsx", index=False)